In [1]:
from pynq import Overlay
Overlay("base.bit").download()

In [2]:
from time import sleep
import math
from pynq.board import Button
from pynq.iop import PMODA
from pynq.iop import PMOD_GROVE_G4
from pynq.iop import Grove_IMU
sense = Grove_IMU(PMODA, PMOD_GROVE_G4)

In [3]:
print('start')
btns = [Button(index) for index in range(4)]
while (btns[3].read()==0):
    if (btns[0].read()==1):
        print('button down')
        sleep(.3)
print('done')

start
button down
button down
done


In [4]:
def getAngleDeg(x,y,mod):
    h=math.sqrt(x*x + y*y)
    rawAng=math.asin(y/h) / (2*math.pi) *360
    if (x>=0 and y>=0):
        adjAng = rawAng
    elif (x<0 and y>=0):
        adjAng = 180-rawAng
    elif (x>=0 and y<0):
        adjAng = 360+rawAng
    elif (x<0 and y<0):
        adjAng = 180-rawAng
    else:
        adjAng = rawAng
        print('error in angle calcs')
        return 0;
    finalAng= adjAng + mod;
    return finalAng
    
def get_tilt_heading(data,altdata):
    [ax, ay, _] = altdata[0:3]
    [mx, my, mz] = data[0:3]
    try:
        pitch = math.asin(-ax)
        roll = math.asin(ay / math.cos(pitch))
    except ZeroDivisionError:
        raise RuntimeError("Value out of range or device not connected.")

    xh = mx * math.cos(pitch) + mz * math.sin(pitch)
    yh = mx * math.sin(roll) * math.sin(pitch) + \
    my * math.cos(roll) - mz * math.sin(roll) * math.cos(pitch)
    _ = -mx * math.cos(roll) * math.sin(pitch) + \
    my * math.sin(roll) + mz * math.cos(roll) * math.cos(pitch)
    tilt_heading = 180 * math.atan2(yh, xh) / math.pi
    if yh < 0:
        tilt_heading += 360
    return float("{0:.2f}".format(tilt_heading))

In [5]:
def azimuth(x,y,z):
    return (180/math.pi * math.atan2(y, x)) % 360
def altitude(x,y,z):
    h = math.hypot(y, x)
    return 180/math.pi * math.atan2(z, h)


In [6]:
print('start')
sense.reset()
data=sense.get_compass()
heading=sense.get_heading()
altdata=sense.get_accl()
print(data)
while (btns[3].read()==0):
    sense.reset()
    data=sense.get_compass()
    heading=get_tilt_heading(data, altdata)
    print(data)
    print(getAngleDeg(data[0],data[1],0))
    print(heading)
    #print(altitude(data[0],data[1],data[2]))
    sleep(2)
print('done')

start
[8.79, 16.7, 11.13]
[8.5, 16.41, 11.43]
62.61688946201157
64.4
[8.5, 16.11, 11.13]
62.182858299925044
63.94
[8.79, 16.7, 10.55]
62.24004538661269
63.85
[8.5, 16.41, 10.84]
62.61688946201157
64.31
[6.74, 16.99, 11.43]
68.36157715380388
70.2
[5.27, 15.82, 11.43]
71.57591705586165
73.59
[4.98, 14.94, 11.72]
71.56505117707799
73.76
[3.81, 14.94, 11.72]
75.69336788565731
77.93
[3.22, 14.36, 11.72]
77.36139803643285
79.7
[2.64, 14.65, 11.72]
79.78466197655426
82.09
[1.76, 14.06, 11.13]
82.86495287747643
85.15
[1.17, 14.06, 11.43]
85.24310335362583
87.59
[0.88, 13.18, 11.13]
86.18015461688101
88.61
[0.59, 13.18, 11.72]
87.43687732285817
89.99
[0.59, 12.6, 11.13]
87.31906066861018
89.85
[0.0, 12.3, 11.13]
90.0
92.57
[0.29, 12.6, 11.13]
88.68152039441757
91.2
[0.29, 12.6, 11.43]
88.68152039441757
91.27
[-1.17, 10.25, 12.01]
96.5119191086237
99.68
[-0.88, 10.55, 12.3]
94.76813618236373
97.97
[-1.46, 10.55, 12.6]
97.87904026112616
101.07
[-1.76, 8.5, 12.6]
101.69828472496857
105.48
[-1.46, 

In [ ]:
print(azimuth(data[0],data[1],data[2]))